In [1]:
import pandas as pd
import numpy as np

from ase import Atoms
import ase.visualize

import sklearn
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
structures = pd.read_csv("data/structures.csv")

### Visualization of molecules

In [3]:
def view3d(df, struct_name):
    """
    Returns: A 3D representation of the molecule
    df: pandas dataframe containing structures
    struct_name: nolecule_name to be viewed
    """
    molecule_df = df[df['molecule_name'] == struct_name]
    atoms = molecule_df['atom'].values.tolist()
    coords = molecule_df[['x', 'y', 'z']].values.tolist()
    
    molecule = Atoms(symbols=atoms, positions=coords)
    
    print("Molecule name: {}".format(struct_name))
    print("Atoms: {}".format(atoms))
    return ase.visualize.view(molecule, viewer="x3d")

In [4]:
#Example molecule
view3d(structures, "dsgdb9nsd_002116")

Molecule name: dsgdb9nsd_002116
Atoms: ['O', 'C', 'N', 'C', 'C', 'C', 'O', 'H', 'H', 'H']


### EDA

In [5]:
train.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095


In [6]:
structures.head(15)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [7]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4659076,dsgdb9nsd_000004,2,0,2JHC
1,4659077,dsgdb9nsd_000004,2,1,1JHC
2,4659078,dsgdb9nsd_000004,2,3,3JHH
3,4659079,dsgdb9nsd_000004,3,0,1JHC
4,4659080,dsgdb9nsd_000004,3,1,2JHC


In [8]:
print("Shape of train data: {}".format(train.shape))
print("Shape of test data: {}".format(test.shape))
print("Types of atoms: {}".format(structures['atom'].unique()))
print("Coupling types: {}".format(train['type'].unique()))
print("Number of molecules in training data: {}".format(train['molecule_name'].nunique()))
print("Number of molecules in test data: {}".format(test['molecule_name'].nunique()))


Shape of train data: (4659076, 6)
Shape of test data: (2505190, 5)
Types of atoms: ['C' 'H' 'N' 'O' 'F']
Coupling types: ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']
Number of molecules in training data: 85012
Number of molecules in test data: 45777


In [9]:
#Average scalar coupling constant by type
train.groupby("type")['scalar_coupling_constant'].agg(['mean', 'std', 'count'])

,mean,std,count
type,,,
1JHC,94.973472,18.290334,709133
1JHN,47.492087,10.897996,43680
2JHC,-0.273447,4.520720,1140867
2JHH,-10.283040,3.983509,377988
2JHN,3.125678,3.676290,119059
3JHC,3.688640,3.075859,1511207
3JHH,4.771004,3.706132,590529
3JHN,0.991046,1.320717,166613


### Merging dataframes


In [10]:
def merge_df_with_structure(df, structure):
    """
    Returns: df containing the training/testing data with structural data
    df: train or test set, as given in the csv file
    structure: structures df as given in the csv file
    """
    df = pd.merge(df, structures, how='left', 
                 left_on = ['molecule_name', 'atom_index_0'],
                 right_on = ['molecule_name', 'atom_index'])

    df =  df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': 'atom_0', 'x': 'atom_index_0_x', 'y': 'atom_index_0_y', 'z': 'atom_index_0_z'})

    df = pd.merge(df, structures, how='left', 
                 left_on = ['molecule_name', 'atom_index_1'],
                 right_on = ['molecule_name', 'atom_index'])

    df =  df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': 'atom_1', 'x': 'atom_index_1_x', 'y': 'atom_index_1_y', 'z': 'atom_index_1_z'})
    
    #Reorder columns so that scalar coupling constant is in the back, if test set
    try:
        cols = df.columns.tolist()
        cols.append(cols.pop(cols.index('scalar_coupling_constant')))
        df = df.reindex(columns=cols)
    except:
        pass
    
    return df


In [11]:
trainx = merge_df_with_structure(train, structures)
testx= merge_df_with_structure(test, structures)

In [12]:
trainx.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,atom_index_0_x,atom_index_0_y,atom_index_0_z,atom_1,atom_index_1_x,atom_index_1_y,atom_index_1_z,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,84.8074


In [13]:
testx.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,atom_index_0_x,atom_index_0_y,atom_index_0_z,atom_1,atom_index_1_x,atom_index_1_y,atom_index_1_z
0,4659076,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4659077,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4659078,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4659079,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4659080,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


### Encoding categorical variables


In [14]:
#One Hot Encoding
dd = trainx.copy()

#Drop columns we don't want to use for prediction
dd = dd.drop(['id', 'molecule_name'], axis=1)


col_transformer = make_column_transformer(
                (['type', 'atom_0', 'atom_1'], OneHotEncoder()),
                remainder='passthrough')

transformed_data = col_transformer.fit_transform(dd)

print("Types of 'atom_0': {}".format(dd['atom_0'].unique()))
print("Types of 'atom_1': {}".format(dd['atom_1'].unique()))
print("Types of 'interactions': {}".format(dd['type'].unique()))

transformed_data[3]

C:\Users\Ruifan\Anaconda3\envs\chem\lib\site-packages\sklearn\compose\_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


Types of 'atom_0': ['H']
Types of 'atom_1': ['C' 'H' 'N']
Types of 'interactions': ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  4.00000000e+00,  2.15041600e-03, -6.03131760e-03,
        1.97612040e-03, -5.23813635e-01,  1.43793264e+00,  9.06397294e-01,
       -1.12543000e+01])

### Dummy Perceptron


In [15]:
def evaluate(predicted, labels):
    """
    Returns: log of mean absolute error (right now overall, not seperated by type and averaged)
    predicted: list of predicted scalar coupling constants
    labels: list actual scalar coupling constants
    """
    n = len(predicted)
    sum_errs = sum(abs(predicted - labels))
    score = np.log(1/n * sum_errs)
    #print(sum_errs/n)
    return score

In [16]:
#Split train and test 
dat1 = transformed_data.copy()

X = [i[:-1] for i in dat1]
Y = [i[-1] for i in dat1]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [17]:
per = MLPRegressor(hidden_layer_sizes=(100,), verbose=True, early_stopping=True)
per.fit(x_train, y_train)

Iteration 1, loss = 40.27422803
Validation score: 0.956427
Iteration 2, loss = 26.33429927
Validation score: 0.957564
Iteration 3, loss = 25.98846717
Validation score: 0.958075
Iteration 4, loss = 25.45313398
Validation score: 0.958930
Iteration 5, loss = 24.99589990
Validation score: 0.959802
Iteration 6, loss = 24.75735868
Validation score: 0.959134
Iteration 7, loss = 24.55670222
Validation score: 0.959781
Iteration 8, loss = 24.40324216
Validation score: 0.960544
Iteration 9, loss = 24.28880727
Validation score: 0.960663
Iteration 10, loss = 24.15500615
Validation score: 0.960541
Iteration 11, loss = 24.06024574
Validation score: 0.961091
Iteration 12, loss = 23.97122261
Validation score: 0.961314
Iteration 13, loss = 23.88877112
Validation score: 0.960856
Iteration 14, loss = 23.82270358
Validation score: 0.961468
Iteration 15, loss = 23.75602860
Validation score: 0.961661
Iteration 16, loss = 23.69897583
Validation score: 0.961397
Iteration 17, loss = 23.65112445
Validation score

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [18]:
y_predicted = per.predict(x_test)

evaluate(y_predicted, y_test)

1.2857512714656132

In [19]:
%reset array
per = MLPRegressor(hidden_layer_sizes=(500,), verbose=True, early_stopping=True)
per.fit(x_train, y_train)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Iteration 1, loss = 32.65077730
Validation score: 0.958453
Iteration 2, loss = 24.63160324
Validation score: 0.960591
Iteration 3, loss = 23.77583625
Validation score: 0.961519
Iteration 4, loss = 23.21486655
Validation score: 0.962707
Iteration 5, loss = 22.78898685
Validation score: 0.963156
Iteration 6, loss = 22.43755923
Validation score: 0.962586
Iteration 7, loss = 22.12149855
Validation score: 0.963266
Iteration 8, loss = 21.86189093
Validation score: 0.964497
Iteration 9, loss = 21.61087560
Validation score: 0.965053
Iteration 10, loss = 21.40896095
Validation score: 0.965306
Iteration 11, loss = 21.21997743
Validation score: 0.965351
Iteration 12, loss = 21.04677274
Validation score: 0.964615
Iteration 13, loss = 20.91852989
Validation score: 0.965384
Iteration 14, loss = 20.77152363
Validation score: 0.966295
Iteration 15, loss = 20.66044098
Validation score: 0.964792
Iteration 16, loss = 20.53873026
Validation s

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(500,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [20]:
y_predicted = per.predict(x_test)

evaluate(y_predicted, y_test)

1.2094644152358631

In [21]:
%reset array
per = MLPRegressor(hidden_layer_sizes=(500,500,200,), verbose=True, early_stopping=True)
per.fit(x_train, y_train)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Iteration 1, loss = 25.24158856
Validation score: 0.965322
Iteration 2, loss = 20.50802431
Validation score: 0.967917
Iteration 3, loss = 18.90822239
Validation score: 0.970138
Iteration 4, loss = 17.98921846
Validation score: 0.969790
Iteration 5, loss = 17.36310849
Validation score: 0.971697
Iteration 6, loss = 16.81971084
Validation score: 0.972639
Iteration 7, loss = 16.35648575
Validation score: 0.973469
Iteration 8, loss = 15.98537453
Validation score: 0.974043
Iteration 9, loss = 15.59875422
Validation score: 0.974392
Iteration 10, loss = 15.35332404
Validation score: 0.975274
Iteration 11, loss = 15.06697804
Validation score: 0.974850
Iteration 12, loss = 14.82259862
Validation score: 0.975699
Iteration 13, loss = 14.63838757
Validation score: 0.975386
Iteration 14, loss = 14.44716527
Validation score: 0.975974
Iteration 15, loss = 14.27406231
Validation score: 0.976894
Iteration 16, loss = 14.10540499
Validation s

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(500, 500, 200), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [22]:
y_predicted = per.predict(x_test)

evaluate(y_predicted, y_test)

0.9701033780590341